In [1]:
#on_kaggle = True
on_kaggle = False

#TRAIN_PREDICT = 'predict'
TRAIN_PREDICT = 'train'

if not on_kaggle:
    import os
    os.environ["CUDA_VISIBLE_DEVICES"]="3"

SEED = 42

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import gc
import os
from scipy.stats import spearmanr
from math import floor, ceil
import random

from unidecode import unidecode
import re
#import gensim
#import string

import tensorflow as tf
import tensorflow.keras.backend as K
#import tensorflow_hub as hub

#fix bug with using CuDNNLSTM
#gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)

from transformers import *

np.set_printoptions(suppress=True)

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed_all(seed)
    #torch.backends.cudnn.deterministic = True
    tf.random.set_seed(seed)
seed_everything(SEED)

#### 1. Read data and tokenizer

Read tokenizer and data, as well as defining the maximum sequence length that will be used for the input to Bert (maximum is usually 512 tokens)

In [3]:
if on_kaggle:
    PATH = '../input/google-quest-challenge/'
    BERT_PATH = '../input/bert-base-uncased-huggingface-transformer/'
    CKPT_LOAD_PATH = ['../input/google-quest-qa-labeling-checkpoints/fold0-epoch3.h5py']
    tokenizer = BertTokenizer.from_pretrained(BERT_PATH+'bert-base-uncased-vocab.txt')
else:##offline
    PATH = '../data/'
    BERT_PATH = '../model/bert-base-uncased'
    CKPT_SAVE_PATH = '../checkpoint/bert-base-uncased-v5/'
    CKPT_LOAD_PATH = ['../checkpoint/bert-base-uncased/']
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

MAX_SEQUENCE_LENGTH = 512

df_train = pd.read_csv(PATH+'train.csv')
df_test = pd.read_csv(PATH+'test.csv')
df_sub = pd.read_csv(PATH+'sample_submission.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

output_categories = list(df_train.columns[11:])
input_categories = list(df_train.columns[[1,2,5,9,10]])#9:category, 10:host
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)

train shape = (6079, 41)
test shape = (476, 11)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer', 'category', 'host']


In [47]:
#read USE sentence embedding model
if on_kaggle:
    pass
else:
    os.environ["TFHUB_CACHE_DIR"]="../data/"
    use_embed = hub.load("https://hub.tensorflow.google.cn/google/universal-sentence-encoder-large/5")

In [46]:
# embeddings = use_embed([
#     "The quick brown fox jumps over the lazy dog. I am a sentence for which I would like to get its embedding"
# ])
# embeddings.shape==[1, 512]

#### 2. Preprocessing functions

These are some functions that will be used to preprocess the raw text data into useable Bert inputs.

In [5]:
##text cleaning 1
import html
for col in ['question_title', 'question_body', 'answer']:
    df_train[col] = df_train[col].apply(html.unescape)
    df_test[col] = df_test[col].apply(html.unescape)

In [150]:
def slide_window_sentence(text):
    """text: raw text full
    
    return: list of string(sentence), ['sentence 1', ..., 'sentence N']
    """
    tokens = tokenizer.encode_plus(text, None, 
                               add_special_tokens=True, max_length=None, 
                               truncation_strategy='longest_first')
    a = tokens['input_ids']
    
    t_len = len(a)
    s_len = 8 #sentence/window length
    overlap = 3 #how many words overlap
    num_sentence = 256 #max number of sentences, if padding or truncate
    #print(t_len, s_len)
    sentences = []
    i = 0
    while True:
        if i==0:
            #print(a[0:s_len])
            sentence = tokenizer.decode(a[0:s_len], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        else:
            #print(a[i*s_len-overlap*i:(i+1)*s_len-overlap*i])
            sentence = tokenizer.decode(a[i*s_len-overlap*i:(i+1)*s_len-overlap*i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        sentences.append(sentence)
        i += 1
        if i*s_len-overlap*i>t_len:
            break
    if len(sentences)<num_sentence:
        to_pad = ['']*(num_sentence-len(sentences))
        sentences.extend(to_pad)
    else:
        sentences = sentences[:num_sentence]
    #return sentences
    return use_embed(sentences)

In [151]:
idx = 1
t,q = df_train.loc[idx, 'question_title'], df_train.loc[idx, 'question_body']
print(t+' '+q+'\n'+'='*40)
#print(' '.join(tokenizer.ids_to_tokens[i] for i in tokens['input_ids']))
#[tokenizer.ids_to_tokens[i] for i in tokens['input_ids']]

#sentences = slide_window_sentence([2054]*6)
sentences = slide_window_sentence(tokens['input_ids'])
print(len(sentences))
sentences

What is the distinction between a city and a sprawl/metroplex... between downtown and a commercial district? I am trying to understand what kinds of places the spam values on p 231 refer to in the 5th Edition main book for Shadowrun.

Per p 15, a sprawl is a plex, a plex is a "metropolitan complex, short for metroplex". Per Google a metroplex is " a very large metropolitan area, especially one that is an aggregation of two or more cities".  A city downtown and sprawl downtown would tend to have similar densities, but for some reason the sprawl (which includes suburbs?) has a higher spam zone noise rating (p 231).  Similarly, I'd think of a downtown as being more dense and noisy (e.g. Office buildings and street vendors) than a commercial district, e.g. an outdoor mall.  The noise ratings make me think that I am thinking about this incorrectly. What is a better way of thinking of them?

256


<tf.Tensor: id=6712782, shape=(256, 512), dtype=float32, numpy=
array([[ 0.00094261,  0.02564181,  0.00950498, ..., -0.0003988 ,
        -0.12570222,  0.03042049],
       [ 0.049396  , -0.0593385 ,  0.04293361, ..., -0.00779004,
        -0.016631  , -0.00771036],
       [ 0.09390454,  0.00138844, -0.02308637, ..., -0.03482159,
        -0.0106916 , -0.04018098],
       ...,
       [-0.05199139,  0.01440313, -0.00282085, ...,  0.00280919,
        -0.10282782, -0.02261897],
       [-0.05199137,  0.01440313, -0.00282086, ...,  0.00280921,
        -0.10282779, -0.02261897],
       [-0.05199137,  0.01440313, -0.00282086, ...,  0.00280921,
        -0.10282779, -0.02261897]], dtype=float32)>

In [26]:
# # add token
# #1. title + [SPECIAL_TOKEN] + question
# special_tokens_dict = {'additional_special_tokens': ['[SPECIAL_TOKEN]']}
# num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
# num_added_toks, len(tokenizer)

In [6]:
def count_special_char(x):
    special_character = re.compile(r'[^A-Za-z\.\-\?\!\,\#\@\% ]', re.IGNORECASE)
    x_ascii = unidecode(x)
    s = special_character.findall(x_ascii)
    #print(s)
    c = len(s)
    return c

def count_cap_char(x):
    c = sum(1 for l in x if l.isupper())
    return c

def count_unique_words(x):
    """returns a ratio"""
    special_character = re.compile(r'[^A-Za-z0-9]', re.IGNORECASE)
    a = [w for w in special_character.split(x) if w!='']
    r = len(set(a))/len(a)
    return r

#print(t)
#count_cap_char(t), len(t)
#count_special_char(t), len(t)
#string.printable

#from textblob import TextBlob
# print(t.lower())
# print('='*40)
# print(TextBlob(t.lower()).correct())

In [7]:
l = df_train.category.unique().tolist()
category2index = dict([(l[i],i+1) for i in range(len(l))])
category2index['UNK'] = 0

l = df_train.host.unique().tolist()
host2index = dict([(l[i],i+1) for i in range(len(l))])
host2index['UNK'] = 0

category2index

{'LIFE_ARTS': 1,
 'CULTURE': 2,
 'SCIENCE': 3,
 'STACKOVERFLOW': 4,
 'TECHNOLOGY': 5,
 'UNK': 0}

In [160]:
def _convert_to_transformer_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)
    
    NOTE: USE Head + Tail truncation
    """
    
    def return_id(str1, str2, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=None,
            truncation_strategy=truncation_strategy)
        
        input_ids =  inputs["input_ids"]
        input_masks = [1] * len(input_ids)
        input_segments = inputs["token_type_ids"]
        if len(input_ids)>length:#Head + Tail truncate
            input_ids = input_ids[:128] + input_ids[-384:]
            input_masks = input_masks[:128] + input_masks[-384:]
            input_segments = input_segments[:128] + input_segments[-384:]
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]
    
    input_ids_q, input_masks_q, input_segments_q = return_id(
        title + ' ' + question, None, 'longest_first', max_sequence_length)
#     input_ids_q, input_masks_q, input_segments_q = return_id(
#         title + ' [SPECIAL_TOKEN] ' + question, None, 'longest_first', max_sequence_length)
    
    input_ids_a, input_masks_a, input_segments_a = return_id(
        answer, None, 'longest_first', max_sequence_length)
    
    return [input_ids_q, input_masks_q, input_segments_q,
            input_ids_a, input_masks_a, input_segments_a]

def compute_input_arrays2(df, columns, tokenizer):
    """for USE model input sentences with sliding window trick
    """
    input_sentences_q, input_sentences_a = [], []
    for row_id, instance in tqdm(df[columns].iterrows()):
        #if row_id>100:
        #    break
        t, q, a = instance.question_title, instance.question_body, instance.answer
        sentences_q = slide_window_sentence(t+' '+q)
        sentences_a = slide_window_sentence(a)
        input_sentences_q.append([sentences_q])
        input_sentences_a.append([sentences_a])
    return [tf.concat(input_sentences_q, axis=0), 
            tf.concat(input_sentences_a, axis=0)
           ]#(N, 32, 64) for raw sentence #(N, 32, 512) for embed


def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    input_ids_q, input_masks_q, input_segments_q = [], [], []
    input_ids_a, input_masks_a, input_segments_a = [], [], []
    special_char_q, special_char_a = [], []
    cap_char_q, cap_char_a = [], []
    unique_words_q, unique_words_a = [], []
    category_index, host_index = [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer
        
        ids_q, masks_q, segments_q, ids_a, masks_a, segments_a = \
        _convert_to_transformer_inputs(t, q, a, tokenizer, max_sequence_length)
        
        input_ids_q.append(ids_q)
        input_masks_q.append(masks_q)
        input_segments_q.append(segments_q)

        input_ids_a.append(ids_a)
        input_masks_a.append(masks_a)
        input_segments_a.append(segments_a)
        
        special_char_q.append(count_special_char(t+q)/len(t+q))
        special_char_a.append(count_special_char(a)/len(a))
        cap_char_q.append(count_cap_char(t+q)/len(t+q))
        cap_char_a.append(count_cap_char(a)/len(a))
        unique_words_q.append(count_unique_words(t+q))
        unique_words_a.append(count_unique_words(a))
        
        category, host = instance.category, instance.host
        category_index.append([category2index.get(category, 0)])
        host_index.append([host2index.get(host, 0)])
        
        #USE embed
        #_q_embed, _a_embed = use_embed([t+' '+q]), use_embed([a])#shape=(1,512)
#         _q_embed, _a_embed = slide_window_sentence(ids_q), slide_window_sentence(ids_a)#shape=(128,512), 128 sentences
#         q_embed.append(_q_embed.numpy())#(n_sample, 128, 512)
#         a_embed.append(_a_embed.numpy())
        
    return [np.asarray(input_ids_q, dtype=np.int32), 
            np.asarray(input_masks_q, dtype=np.int32), 
            np.asarray(input_segments_q, dtype=np.int32),
            np.asarray(input_ids_a, dtype=np.int32), 
            np.asarray(input_masks_a, dtype=np.int32), 
            np.asarray(input_segments_a, dtype=np.int32),
            np.asarray([special_char_q, special_char_a, cap_char_q, cap_char_a, unique_words_q, unique_words_a], dtype=np.float32).T, 
            np.asarray(category_index, dtype=np.int32),
            np.asarray(host_index, dtype=np.int32),
#             np.asarray(q_embed, dtype=np.float32),
#             np.asarray(a_embed, dtype=np.float32)
           ]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

#### 3. Create model

`compute_spearmanr()` is used to compute the competition metric for the validation set
<br><br>
`CustomCallback()` is a class which inherits from `tf.keras.callbacks.Callback` and will compute and append validation score and validation/test predictions respectively, after each epoch.
<br><br>
`bert_model()` contains the actual architecture that will be used to finetune BERT to our dataset. It's simple, just taking the sequence_output of the bert_layer and pass it to an AveragePooling layer and finally to an output layer of 30 units (30 classes that we have to predict)
<br><br>
`train_and_predict()` this function will be run to train and obtain predictions

In [153]:
def compute_spearmanr_ignore_nan(trues, preds):
    rhos = []
    for tcol, pcol in zip(np.transpose(trues), np.transpose(preds)):
        rhos.append(spearmanr(tcol, pcol).correlation)
    return np.nanmean(rhos)


class CustomCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, valid_data, test_data=None, batch_size=16, fold=None, stage2=False):

        self.valid_inputs = valid_data[0]
        self.valid_outputs = valid_data[1]
        self.test_inputs = test_data
        
        self.batch_size = batch_size
        self.fold = fold
        self.stage2 = stage2
        
    def on_train_begin(self, logs={}):
        self.valid_predictions = []
        self.test_predictions = []
        
    def on_epoch_end(self, epoch, logs={}):
        self.valid_predictions.append(
            self.model.predict(self.valid_inputs, batch_size=self.batch_size))
        
        rho_val = compute_spearmanr_ignore_nan(
            self.valid_outputs, np.average(self.valid_predictions, axis=0))
        
        print("\nvalidation rho: %.4f" % rho_val)
        
        if self.fold is not None and epoch>1:
            if self.stage2:
                self.model.save_weights(CKPT_SAVE_PATH+f'fold{fold}-epoch{epoch}-stage2.h5py')
            else:
                self.model.save_weights(CKPT_SAVE_PATH+f'fold{fold}-epoch{epoch}.h5py')
        
#         self.test_predictions.append(
#             self.model.predict(self.test_inputs, batch_size=self.batch_size)
#         )

def use_model():
    q_sentences = tf.keras.layers.Input((256,512), dtype=tf.float32)#(N, num_sentences, features)
    a_sentences = tf.keras.layers.Input((256,512), dtype=tf.float32)
    #qa = tf.keras.layers.Concatenate(axis=-1)([q_sentences, a_sentences])
    q1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True, 
                                                            activation='tanh', recurrent_activation='sigmoid', 
                                                            recurrent_dropout=0, unroll=False, use_bias=False
                                                            ))(q_sentences)
    q2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, 
                                                            activation='tanh', recurrent_activation='sigmoid', 
                                                            recurrent_dropout=0, unroll=False, use_bias=False
                                                            ))(q1)
    a1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True, 
                                                            activation='tanh', recurrent_activation='sigmoid', 
                                                            recurrent_dropout=0, unroll=False, use_bias=False
                                                            ))(a_sentences)
    a2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, 
                                                            activation='tanh', recurrent_activation='sigmoid', 
                                                            recurrent_dropout=0, unroll=False, use_bias=False
                                                            ))(a1)
    
    q1_avgpool = tf.keras.layers.GlobalAveragePooling1D()(q1)
    q2_avgpool = tf.keras.layers.GlobalAveragePooling1D()(q2)
    a1_avgpool = tf.keras.layers.GlobalAveragePooling1D()(a1)
    a2_avgpool = tf.keras.layers.GlobalAveragePooling1D()(a2)
    
    x = tf.keras.layers.Concatenate()([q1_avgpool, q2_avgpool, a1_avgpool, a2_avgpool,])
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    x = tf.keras.layers.Dense(30, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=[q_sentences, a_sentences, ], 
                                  outputs=x)
    
    return model
    

def bert_model():
    q_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    feats = tf.keras.layers.Input((6,), dtype=tf.float32)#set dim of additional numeric feats
    category_index = tf.keras.layers.Input((1,), dtype=tf.int32)
    host_index = tf.keras.layers.Input((1,), dtype=tf.int32)

    config = BertConfig() # print(config) to see settings
    config.output_hidden_states = False # Set to True to obtain hidden states
    # caution: when using e.g. XLNet, XLNetConfig() will automatically use xlnet-large config
    
    # normally ".from_pretrained('bert-base-uncased')", but because of no internet, the 
    # pretrained model has been downloaded manually and uploaded to kaggle.
    #base_model = bert_base_model()
    #base_model.load_weights(CKPT_SAVE_PATH+f'fold{fold}-epoch3.h5py')
    #bert_weights = base_model.layers[8].get_weights()
    if on_kaggle:
        bert_layer = TFBertModel.from_pretrained(BERT_PATH+'bert-base-uncased-tf_model.h5', config=config)
    else:
        bert_layer = TFBertModel.from_pretrained('bert-base-uncased', config=config)
    #bert_layer.set_weights(bert_weights)
    #bert_layer.trainable = False
    
    # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
    q_embedding = bert_layer(q_id, attention_mask=q_mask, token_type_ids=q_atn)[0]
    a_embedding = bert_layer(a_id, attention_mask=a_mask, token_type_ids=a_atn)[0]
    
    #q_embedding = tf.keras.layers.SpatialDropout1D(0.2)(q_embedding)
    #a_embedding = tf.keras.layers.SpatialDropout1D(0.2)(a_embedding)
    
    qa_embedding = tf.keras.layers.Concatenate(axis=1)([q_embedding, a_embedding])#(None,512,768)
    qa1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True, 
                                                            activation='tanh', recurrent_activation='sigmoid', 
                                                            recurrent_dropout=0, unroll=False, use_bias=False
                                                            ))(qa_embedding)
    qa2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, 
                                                            activation='tanh', recurrent_activation='sigmoid', 
                                                            recurrent_dropout=0, unroll=False, use_bias=False
                                                            ))(qa1)
    
    qa1_avgpool = tf.keras.layers.GlobalAveragePooling1D()(qa1)
    qa2_avgpool = tf.keras.layers.GlobalAveragePooling1D()(qa2)
    #qa_maxpool = tf.keras.layers.GlobalMaxPool1D()(qa)
    q_avgpool = tf.keras.layers.GlobalAveragePooling1D()(q_embedding)
    a_avgpool = tf.keras.layers.GlobalAveragePooling1D()(a_embedding)
    
    category_embed_layer = tf.keras.layers.Embedding(6, 8, input_length=1)
    host_embed_layer = tf.keras.layers.Embedding(64, 8, input_length=1)
    cat_embed = category_embed_layer(category_index)
    host_embed = host_embed_layer(host_index)
    cat_embed = tf.keras.layers.Flatten()(cat_embed)
    host_embed = tf.keras.layers.Flatten()(host_embed)
    
    #x = tf.keras.layers.Concatenate()([q, a, feats, cat_embed, host_embed])
    x = tf.keras.layers.Concatenate()([qa1_avgpool, qa2_avgpool, q_avgpool, a_avgpool,
                                       feats, cat_embed, host_embed,])
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    x = tf.keras.layers.Dense(30, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn, a_id, a_mask, a_atn, 
                                          feats, category_index, host_index, ], 
                                  outputs=x)
    
    return model


def train_and_predict(model, train_data, valid_data, test_data, 
                      learning_rate, epochs, batch_size, loss_function, fold, stage2=False):
    """
    multipliers = {'dense_1': 0.5, 'dense_2': 0.4}
    optimizer = LearningRateMultiplier(tf.keras.optimizers.Adam, 
                                        lr_multiplier=multipliers, learning_rate=learning_rate)
    
    """
    custom_callback = CustomCallback(
        valid_data=(valid_data[0], valid_data[1]), 
        test_data=test_data,
        batch_size=batch_size,
        fold=fold, stage2=stage2)

    #decay_steps = 1000
    #lr_decayed_fn = tf.keras.experimental.CosineDecay(learning_rate, decay_steps)
    #clr = CyclicLR(base_lr=2e-5, max_lr=5e-5, step_size=2000., mode='triangular')

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)#clipvalue=0.5
    model.compile(loss=loss_function, optimizer=optimizer)
    model.fit(train_data[0], train_data[1], epochs=epochs, 
              batch_size=batch_size, callbacks=[custom_callback])
    
    return custom_callback

def predict(model, test_data, load_weights_path):
    model.load_weights(load_weights_path)
    return model.predict(test_data, batch_size=BATCH_SIZE)

In [29]:
# config = BertConfig()
# #print(config)
# config.output_hidden_states = False
# model = bert_model()
# model.summary()


#base_model.load_weights('../checkpoint/bert-base-uncased-v3/fold0-epoch3.h5py')

#bert_weights = base_model.layers[8].get_weights()

#new_bert_layer = TFBertModel.from_pretrained('bert-base-uncased', config=config)
#new_bert_layer.set_weights(bert_weights)

#### 4. Obtain inputs and targets, as well as the indices of the train/validation splits

In [161]:
if TRAIN_PREDICT == 'train':
    outputs = compute_output_arrays(df_train, output_categories)
    #inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
    inputs = compute_input_arrays2(df_train, input_categories, tokenizer)

ResourceExhaustedError: OOM when allocating tensor with shape[6079,256,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ConcatV2] name: concat

In [167]:
inputs[0].shape

TensorShape([101, 256, 512])

In [163]:
import pickle

if TRAIN_PREDICT == 'train':
    if not os.path.isfile('../data/gkf%d.pkl'%SEED):
        print('Create gkf')
        gkf = GroupKFold(n_splits=5).split(X=df_train.question_body, groups=df_train.question_body)
        with open('../data/gkf%d.pkl'%SEED, 'wb') as f:
            pickle.dump(list(gkf), f)
    else:
        print('Load gkf')
        with open('../data/gkf%d.pkl'%SEED, 'rb') as f:
            gkf = pickle.load(f)

Load gkf


In [14]:
#print(len(inputs), inputs[6].shape, inputs[7], inputs[8])
#inputs[6][:,4:]
# for i,(train_idx,val_idx) in enumerate(gkf):
#     if i==0:
#         break
# train_idx[20:30]
#inputs[8][0].shape

#gkf0 = list(gkf)

#### 5. Training, validation and testing

Loops over the folds in gkf and trains each fold for 5 epochs --- with a learning rate of 1e-5 and batch_size of 8. A simple binary crossentropy is used as the objective-/loss-function. 

In [15]:
NUM_EPOCHS = 4
BATCH_SIZE = 8
LearningRate = 3e-5

In [16]:
def custom_bce_loss(y_true, y_pred):
    """
    nunique = [df_train[col].nunique() for col in output_categories]#count unique values of each column
    weights_dict = {5:0.5, 9:1.0, 17:1.5, 3:0.5}
    weights = [weights_dict[i] for i in nunique]
    """
    weights = tf.convert_to_tensor([1. , 1. , 0.5, 0.5, 0.5, 0.5, 1. , 1. , 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 1. , 1. , 1. , 1. , 1.5,
       0.5, 0.5, 0.5, 1. ], dtype=float)
    weights = weights/K.mean(weights)
    loss = tf.multiply(y_true, K.log(y_pred+K.epsilon())) + tf.multiply((1-y_true), K.log(1-y_pred+K.epsilon()))
    loss = tf.multiply(loss, weights)
    bce_loss = tf.reduce_mean(-loss)
    return bce_loss
#     y_true_clip = K.clip(y_true, K.epsilon(), 1)
#     y_pred_clip = K.clip(y_pred, K.epsilon(), 1)
#     kl_loss = tf.reduce_mean(tf.reduce_sum(y_true_clip * K.log(y_true_clip / y_pred_clip), axis=0))
#     return bce_loss*0.9 + kl_loss*0.1

#y_pred = tf.random.uniform((8, 30))
#y_true = tf.random.uniform((8, 30))

#custom_bce_loss(y_true, y_pred)

In [164]:
## training
## if stage2, modify 2 places
if TRAIN_PREDICT == 'train':
    histories = []
    for fold, (train_idx, valid_idx) in enumerate(gkf):

        # will actually only do 1 fold (out of 5) to compare models
        if fold ==0:#>-1:
            print('========training fold %d========'%fold)
            K.clear_session()
            
            #model = bert_model()
            model = use_model()
            
            #prepare dataset
#             train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
#             train_outputs = outputs[train_idx]
#             valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
#             valid_outputs = outputs[valid_idx]

            train_inputs = [inputs[i].numpy()[train_idx] for i in range(len(inputs))]
            train_outputs = outputs[train_idx]
            valid_inputs = [inputs[i].numpy()[valid_idx] for i in range(len(inputs))]
            valid_outputs = outputs[valid_idx]

            # history contains two lists of valid and test preds respectively:
            #  [valid_predictions_{fold}, test_predictions_{fold}]
            history = train_and_predict(model, 
                              train_data=(train_inputs, train_outputs), 
                              valid_data=(valid_inputs, valid_outputs),
                              test_data=None, 
                              learning_rate=LearningRate, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
                              loss_function=custom_bce_loss, fold=fold, 
                                        stage2=False)#'binary_crossentropy'
            histories.append(history)

# ## training full trainset to lift LB score in the final
# if TRAIN_PREDICT == 'train':
#     histories = []
#     print('========Start training========')
#     K.clear_session()
#     model = bert_model()

#     train_inputs = inputs
#     train_outputs = outputs

#     history = train_and_predict(model, 
#                       train_data=(train_inputs, train_outputs), 
#                       valid_data=None,
#                       test_data=None, 
#                       learning_rate=LearningRate, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
#                       loss_function='binary_crossentropy', fold=fold)

#     histories.append(history)

========training fold 0========


IndexError: index 101 is out of bounds for axis 0 with size 101

In [ ]:
#more ideas todo: 
#1.1 modify model
    #sliding window for sentences --> USE embed --> LSTM
    #https://github.com/google-research/bert/issues/66
    #https://www.kaggle.com/c/google-quest-challenge/discussion/120185#689854
#post-process, transform prediction to rank, to get averaged rank across folds, then assign new predictions
#1.2 OOV words spelling correction
#1.3 train >4 epochs
#2. loss, ranking loss?
#3. add custom new tokens?(e.g stackoverflow)
#4. roberta?alxnet?
#5. RankGauss average folds?
#6. freeze some layers of bert?
"""
CV history
---------------
#### bert-base ####
Epoch 5/15
4856/4863 [============================>.] - ETA: 0s - loss: 0.3091
validation rho: 0.3923
4863/4863 [==============================] - 339s 70ms/sample - loss: 0.3091
------LB=0.346

switch to HuggingFace
----------------------
SEEMS DEPENDS ON THE SEED!!!
----------------------------
t + q[:1/2], q[1/2:], a
same

category + host + t + q, category + host + a
same


CUSTOM LOSS
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3612
validation rho: 0.3989
4863/4863 [==============================] - 711s 146ms/sample - loss: 0.3612

Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3597
validation rho: 0.3998
4863/4863 [==============================] - 724s 149ms/sample - loss: 0.3597

add 2 feats
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3616
validation rho: 0.4049
4863/4863 [==============================] - 718s 148ms/sample - loss: 0.3616

4 feats
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3592
validation rho: 0.4027
4863/4863 [==============================] - 706s 145ms/sample - loss: 0.3591

add cat+host embed dim=16
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3594
validation rho: 0.4005
4863/4863 [==============================] - 698s 144ms/sample - loss: 0.3594

embed dim=8
Epoch 4/4
4856/4863 [============================>.] - ETA: 0s - loss: 0.3592
validation rho: 0.4071
4863/4863 [==============================] - 734s 151ms/sample - loss: 0.3592

add LSTM features
validation rho: 0.4081 --fold0
validation rho: 0.4050 --fold1
validation rho: 0.4138 --fold2
validation rho: 0.4018 --fold3
validation rho: 0.3982 --fold4
LB=0.388, CV=0.405

CV 0.397 to 0.405, LB 0.387 to 0.388 ???


add t+[SPECIAL TOKEN]+q
validation rho: 0.4113 --fold0
validation rho: 0.4014 --fold1
validation rho: 0.4224 --fold2
validation rho: 0.4055 --fold3
validation rho: 0.3961 --fold4
CV=0.407, LB=0.387

--------
epochs progression:

4856/4863 [============================>.] - ETA: 2s - loss: 0.4053
validation rho: 0.3832
4863/4863 [==============================] - 2045s 421ms/sample - loss: 0.4052
Epoch 2/4
4856/4863 [============================>.] - ETA: 2s - loss: 0.3799
validation rho: 0.4016
4863/4863 [==============================] - 1995s 410ms/sample - loss: 0.3799
Epoch 3/4
4856/4863 [============================>.] - ETA: 2s - loss: 0.3680
validation rho: 0.4092
4863/4863 [==============================] - 2002s 412ms/sample - loss: 0.3680
Epoch 4/4
4856/4863 [============================>.] - ETA: 2s - loss: 0.3557
validation rho: 0.4108
4863/4863 [==============================] - 1998s 411ms/sample - loss: 0.3557

"""

#### 6. Process and submit test predictions

First the test predictions are read from the list of lists of `histories`. Then each test prediction list (in lists) is averaged. Then a mean of the averages is computed to get a single prediction for each data point. Finally, this is saved to `submission.csv`

In [12]:
if TRAIN_PREDICT == 'predict':
    
    test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
    
    model = bert_model()
    
    test_predictions = [predict(model, test_inputs, load_weights_path=ckpt_load_path) 
                        for ckpt_load_path in CKPT_LOAD_PATH]
    #test_predictions = [np.average(test_predictions[i], axis=0) for i in range(len(test_predictions))]
    test_predictions = np.mean(test_predictions, axis=0)

    df_sub.iloc[:, 1:] = test_predictions

    df_sub.to_csv('submission.csv', index=False)

NameError: name 'compute_input_arays' is not defined

In [ ]:
df_sub.head(10)

In [ ]:
# ##post process
# cols2process = df_train.columns.tolist()[11:]
# print(len(cols2process))

In [ ]:
# for col in tqdm(cols2process):
#     ###step1
#     quantiles = df_train[col].value_counts()/len(df_train.index)
#     quantiles = quantiles.to_dict()
#     quantiles = {k: v for k, v in sorted(quantiles.items(), key=lambda item: item[0])}
#     ks = list(quantiles.keys())
#     vs = list(quantiles.values())
#     qs = np.cumsum(vs)
#     #print(ks)
#     #print(qs)
#     ###step2
#     qs = np.quantile(df_sub[col], qs)
#     #print(qs)
#     for i in range(len(qs)):
#         if i==0:
#             q = qs[0]
#             df_sub.loc[df_sub[col]<q, col] = ks[i]
#         elif i>0 and i<=len(qs)-1:
#             q0,q1 = qs[i-1], qs[i]
#             df_sub.loc[(df_sub[col]<q1)&(df_sub[col]>=q0), col] = ks[i]

In [ ]:
# df_sub.head(10)

In [ ]:
# df_sub.to_csv('submission.csv', index=False)